In [1]:
import os

from qiskit import BasicAer, execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit

from quantuminspire.credentials import get_authentication
from coreapi.auth import BasicAuthentication
from quantuminspire.qiskit import QI

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#IBMQ.save_account('MY_TOKEN_NUMBER',overwrite=True)  #save your creds
#provider = IBMQ.load_account()
from quantuminspire.credentials import enable_account
from random import getrandbits
import qiskit as q

In [ ]:
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

In [ ]:
def select_encoding(length):
    
    #This stores the states Alice will encode
    alice_bitstring = ""
    # This stores the bases that Alice will prepare the states in
    alice_bases = ""
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        alice_bitstring += (str(getrandbits(1)))
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        alice_bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return alice_bitstring, alice_bases

In [ ]:
def encode(alice_bitstring, alice_bases):
    encoded_qubits = []
    for i in range(len(alice_bitstring)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = q.QuantumCircuit(1,1)

        if alice_bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if alice_bitstring[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif alice_bitstring[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif alice_bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if alice_bitstring[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif alice_bitstring[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

In [ ]:
def select_measurement(length):
    # Similar to before we store the bases that Bob will measure in a list
    bob_bases = ""
    
    for i in range(length):
        # Again we use getrandbits to generate a 0 or 1 randomly
        bob_bases += (str(getrandbits(1)))
        
    # return the list of random bases to measure in
    return bob_bases

In [ ]:
def measure(bob_bases, encoded_qubits, backend):
    # Perform measurement on the qubits send by Alice
    # selected_measurements: 
    # encoded_qubits: list of QuantumCircuits received from Alice
    # backend: IBMQ backend, either simulation or hardware
    
    # Stores the results of Bob's measurements
    bob_bitstring = ''
    
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bob_bases[i] == "0":
            # 0 means we want to measure in Z basis
            qc.measure(0,0)

        elif bob_bases[i] == "1":
            # 1 means we want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = q.execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bitstring += measured_bit 
        
    return bob_bitstring

In [ ]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

In [ ]:
def construct_key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # For the indices where bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

In [ ]:
QI_URL = os.getenv('QI_URL', 'https://api.quantum-inspire.com/')
authentication = get_authentication()
QI.set_authentication(authentication, QI_URL)
print(QI.backends())
backend = QI.get_backend('Spin-2')

In [ ]:
# Step 1: Alice's Prepares encoding basis and choose a random btistring
KEY_LENGTH=260
limit=4
while limit > 0:
    limit-=1
    print('check', limit)
    SECURE_LENGTH = 30
    Alice_bitstring, Alice_bases = select_encoding(KEY_LENGTH+SECURE_LENGTH)
    # Step 2
    Bob_bases = select_measurement(KEY_LENGTH + SECURE_LENGTH)

    # Step 3
    encoded_qubits = encode(Alice_bitstring, Alice_bases)
    # Step 4
    QUANTUM_CHANNEL = encoded_qubits

    # Step 5
    Bob_bitstring = measure(Bob_bases, QUANTUM_CHANNEL, backend)

    # Step 6: Bob uses Alice's announced bases to see where they agreed with his decoding bases
    CLASSICAL_CHANNEL = Alice_bases
    agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL[:SECURE_LENGTH], Bob_bases[:SECURE_LENGTH])
    Alice_secure_check =  construct_key_from_indices(Alice_bitstring[:SECURE_LENGTH], agreeing_bases)
    Bob_secure_check = construct_key_from_indices(Bob_bitstring[:SECURE_LENGTH], agreeing_bases)
    if Alice_secure_check==Bob_secure_check:
        agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, Bob_bases)
        break

# Bob announces where they agreed on encoding and decoding bases.
Bob_key = construct_key_from_indices(Bob_bitstring, agreeing_bases)[SECURE_LENGTH:]
print("Bob's key:", Bob_key)

# Step 7: Alice and Bob construct their keys from 
CLASSICAL_CHANNEL = agreeing_bases
Alice_key = construct_key_from_indices(Alice_bitstring, CLASSICAL_CHANNEL)[SECURE_LENGTH:]
print("Alice's key:", Alice_key)

In [ ]:
import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [ ]:
OriginString="Hey iquHack is fun right?"
encrypted_message = encrypt_message(OriginString, Alice_key)
decrypt=_message = decrypt_message(encrypted_message, Bob_key)
print(decrypt_message)